In [30]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.mixture import GaussianMixture
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
import numpy as np
import extractor
import os
import pickle
from statistics import mean


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#features_extraction
X_Rajesh= features(24,"drive/My Drive/MIES_DATA/Rajesh/log")
X_Manoj= features(26,"drive/My Drive/MIES_DATA/Manoj/")
X_Daiveek= features(18,"drive/My Drive/MIES_DATA/Daiveek/")
X_Sudheer= features(26,"drive/My Drive/MIES_DATA/Sudheer/log")
X_Vamsi= features(68,"drive/My Drive/MIES_DATA/Vamsi/")
X_imposter=features(35,"drive/My Drive/MIES_DATA/imposter/")

In [6]:
#importing Data from pickle files since we have already extracted features and stored in .txt pickle files to save time
pickle_off = open ("drive/My Drive/MIES_DATA/rajesha.txt", "rb")
X_Rajesh= pickle.load(pickle_off)

pickle_off = open ("drive/My Drive/MIES_DATA/daiveeka.txt", "rb")
X_Daiveek = pickle.load(pickle_off)

pickle_off = open ("drive/My Drive/MIES_DATA/manoja.txt", "rb")
X_Manoj= pickle.load(pickle_off)

pickle_off = open ("drive/My Drive/MIES_DATA/sudheera.txt", "rb")
X_Sudheer = pickle.load(pickle_off)

pickle_off = open ("drive/My Drive/MIES_DATA/sudheera.txt", "rb")
X_Sudheer = pickle.load(pickle_off)

pickle_off = open ("drive/My Drive/MIES_DATA/vamsia.txt", "rb")
X_Vamsi= pickle.load(pickle_off)

X_imposter=np.load('drive/My Drive/MIES_DATA/imposter.npy')

In [13]:
#printing sizes of individual dataset
print(X_Rajesh.shape)
print(X_Daiveek.shape)
print(X_Manoj.shape)
print(X_Sudheer.shape)
print(X_Vamsi.shape)
print(X_imposter.shape)

(9092, 63)
(813, 63)
(19983, 63)
(15313, 63)
(15064, 63)
(16554, 63)


In [14]:
#combining all the authenticated users into single variable and define corresponding y variables for imposter and authenticated users
X_auth_final=np.concatenate((X_Rajesh,X_Manoj,X_Vamsi,X_Sudheer,X_Daiveek),axis = 0)
X_imposter=X_imposter
y_auth_final=np.zeros(len(X_auth_final))
y_imposter=np.ones(len(X_imposter))
print(X_auth_final.shape)
print(y_auth_final.shape)
print(X_imposter.shape)
print(y_imposter.shape)

(60265, 63)
(60265,)
(16554, 63)
(16554,)


In [15]:
#concatenate imposter and authenticated data for feature selection
X=np.concatenate((X_auth_final,X_imposter),axis=0)
y=np.concatenate((y_auth_final,y_imposter),axis=0)
print(X.shape)
print(y.shape)

(76819, 63)
(76819,)


In [86]:
#using SFS technique for feature selection
knn = KNeighborsClassifier(n_neighbors=63)
sfs1 = SFS(knn,
           k_features=5,
           forward=True,
           floating=False,
           scoring='accuracy',
           cv=4)

In [ ]:
#fitting X,y
sfs1 = sfs1.fit(X, y)
print(sfs1.k_feature_idx_)

In [37]:
#combining all data in to a variable with only selected features
X=[X_Rajesh[:,(0,1,2,3,60)],X_Manoj[:,(0,1,2,3,60)],X_Vamsi[:,(0,1,2,3,60)],X_Sudheer[:,(0,1,2,3,60)],X_Daiveek[:,(0,1,2,3,60)],X_imposter[:,(0,1,2,3,60)]]

In [38]:
#partioning individual datasets in to 5 parts for each user 
Train_part1=[]
Train_part2=[]
Train_part3=[]
Train_part4=[]
Train_part5=[]
for i in range(6):
  Training_Data = X[i]
  delta = int(Training_Data.shape[0]/5)
  Train_part1.append(Training_Data[0:delta,:])
  Train_part2.append(Training_Data[delta:2*delta,:])
  Train_part3.append(Training_Data[2*delta:3*delta,:])
  Train_part4.append(Training_Data[3*delta:4*delta,:])
  Train_part5.append(Training_Data[4*delta:,:])


In [39]:
#Fold-1 cross Validation

X_test=Train_part1
X_train0=np.concatenate((Train_part2[0],Train_part3[0],Train_part4[0],Train_part5[0]),axis=0)
X_train1=np.concatenate((Train_part2[1],Train_part3[1],Train_part4[1],Train_part5[1]),axis=0)
X_train2=np.concatenate((Train_part2[2],Train_part3[2],Train_part4[2],Train_part5[2]),axis=0)
X_train3=np.concatenate((Train_part2[3],Train_part3[3],Train_part4[3],Train_part5[3]),axis=0)
X_train4=np.concatenate((Train_part2[4],Train_part3[4],Train_part4[4],Train_part5[4]),axis=0)
X_train5=np.concatenate((Train_part2[5],Train_part3[5],Train_part4[5],Train_part5[5]),axis=0)
X_train=np.concatenate((X_train0,X_train1,X_train2,X_train3,X_train4,X_train5),axis=0)
gmm = GaussianMixture(n_components=2)
gmm.fit(X_train)

print("Training Accuracies")
labels_rajesh = gmm.predict(X_train0)
a=np.bincount(labels_rajesh.astype(int))

labels_manoj = gmm.predict(X_train1)
b=np.bincount(labels_manoj.astype(int))

labels_vamsi = gmm.predict(X_train2)
c=np.bincount(labels_vamsi.astype(int))

labels_sudheer = gmm.predict(X_train3)
d=np.bincount(labels_sudheer.astype(int))

labels_daiveek=gmm.predict(X_train4)
e=np.bincount(labels_daiveek.astype(int))

labels_imposter= gmm.predict(X_train5)
f=np.bincount(labels_imposter.astype(int))

x=[]
x.append((np.max(a)/np.sum(a))*100)
x.append((np.max(b)/np.sum(b))*100)
x.append((np.max(c)/np.sum(c))*100)
x.append((np.max(d)/np.sum(d))*100)
x.append((np.max(e)/np.sum(e))*100)
x.append((np.max(f)/np.sum(f))*100)

print(a,x[0])
print(b,x[1])
print(c,x[2])
print(d,x[3])
print(e,x[4])
print(f,x[5])
print("average training accuracy:",mean(x))
print("end")
print()

print("Test Accuracies")
labels_rajesh = gmm.predict(X_test[0])
a=np.bincount(labels_rajesh.astype(int))

labels_manoj = gmm.predict(X_test[1])
b=np.bincount(labels_manoj.astype(int))

labels_vamsi = gmm.predict(X_test[2])
c=np.bincount(labels_vamsi.astype(int))

labels_sudheer = gmm.predict(X_test[3])
d=np.bincount(labels_sudheer.astype(int))

labels_daiveek=gmm.predict(X_test[4])
e=np.bincount(labels_daiveek.astype(int))

labels_imposter= gmm.predict(X_test[5])
f=np.bincount(labels_imposter.astype(int))

x=[]
x.append((np.max(a)/np.sum(a))*100)
x.append((np.max(b)/np.sum(b))*100)
x.append((np.max(c)/np.sum(c))*100)
x.append((np.max(d)/np.sum(d))*100)
x.append((np.max(e)/np.sum(e))*100)
x.append((np.max(f)/np.sum(f))*100)

print(a,x[0])
print(b,x[1])
print(c,x[2])
print(d,x[3])
print(e,x[4])
print(f,x[5])
print("average test accuracy:",mean(x))
print("end")
    

Training Accuracies
[7263   11] 99.84877646411879
[15987] 100.0
[12048     4] 99.96681048788582
[12243     8] 99.9346992082279
[521 130] 80.03072196620585
[    0 13244] 100.0
average training accuracy: 96.63016802107306
end

Test Accuracies
[1817    1] 99.94499449944995
[3996] 100.0
[3009    3] 99.9003984063745
[3060    2] 99.9346832135859
[120  42] 74.07407407407408
[ 401 2909] 87.88519637462235
average test accuracy: 93.62322442801779
end


In [40]:
#Fold-2 cross validation

X_test=Train_part2
X_train0=np.concatenate((Train_part1[0],Train_part3[0],Train_part4[0],Train_part5[0]),axis=0)
X_train1=np.concatenate((Train_part1[1],Train_part3[1],Train_part4[1],Train_part5[1]),axis=0)
X_train2=np.concatenate((Train_part1[2],Train_part3[2],Train_part4[2],Train_part5[2]),axis=0)
X_train3=np.concatenate((Train_part1[3],Train_part3[3],Train_part4[3],Train_part5[3]),axis=0)
X_train4=np.concatenate((Train_part1[4],Train_part3[4],Train_part4[4],Train_part5[4]),axis=0)
X_train5=np.concatenate((Train_part1[5],Train_part3[5],Train_part4[5],Train_part5[5]),axis=0)
X_train=np.concatenate((X_train0,X_train1,X_train2,X_train3,X_train4,X_train5),axis=0)
gmm = GaussianMixture(n_components=2)
gmm.fit(X_train)

print("Training Accuracies")
labels_rajesh = gmm.predict(X_train0)
a=np.bincount(labels_rajesh.astype(int))

labels_manoj = gmm.predict(X_train1)
b=np.bincount(labels_manoj.astype(int))

labels_vamsi = gmm.predict(X_train2)
c=np.bincount(labels_vamsi.astype(int))

labels_sudheer = gmm.predict(X_train3)
d=np.bincount(labels_sudheer.astype(int))

labels_daiveek=gmm.predict(X_train4)
e=np.bincount(labels_daiveek.astype(int))

labels_imposter= gmm.predict(X_train5)
f=np.bincount(labels_imposter.astype(int))

x=[]
x.append((np.max(a)/np.sum(a))*100)
x.append((np.max(b)/np.sum(b))*100)
x.append((np.max(c)/np.sum(c))*100)
x.append((np.max(d)/np.sum(d))*100)
x.append((np.max(e)/np.sum(e))*100)
x.append((np.max(f)/np.sum(f))*100)

print(a,x[0])
print(b,x[1])
print(c,x[2])
print(d,x[3])
print(e,x[4])
print(f,x[5])
print("average training accuracy:",mean(x))
print("end")
print()

print("Test Accuracies")
labels_rajesh = gmm.predict(X_test[0])
a=np.bincount(labels_rajesh.astype(int))

labels_manoj = gmm.predict(X_test[1])
b=np.bincount(labels_manoj.astype(int))

labels_vamsi = gmm.predict(X_test[2])
c=np.bincount(labels_vamsi.astype(int))

labels_sudheer = gmm.predict(X_test[3])
d=np.bincount(labels_sudheer.astype(int))

labels_daiveek=gmm.predict(X_test[4])
e=np.bincount(labels_daiveek.astype(int))

labels_imposter= gmm.predict(X_test[5])
f=np.bincount(labels_imposter.astype(int))

x=[]
x.append((np.max(a)/np.sum(a))*100)
x.append((np.max(b)/np.sum(b))*100)
x.append((np.max(c)/np.sum(c))*100)
x.append((np.max(d)/np.sum(d))*100)
x.append((np.max(e)/np.sum(e))*100)
x.append((np.max(f)/np.sum(f))*100)

print(a,x[0])
print(b,x[1])
print(c,x[2])
print(d,x[3])
print(e,x[4])
print(f,x[5])
print("average test accuracy:",mean(x))
print("end")
    

Training Accuracies
[  19 7255] 99.73879571075062
[    0 15987] 100.0
[    8 12044] 99.93362097577165
[   10 12241] 99.91837401028488
[168 483] 74.19354838709677
[12844   400] 96.97976442162489
average training accuracy: 95.1273505842548
end

Test Accuracies
[   7 1811] 99.61496149614962
[   0 3996] 100.0
[   1 3011] 99.9667994687915
[   3 3059] 99.90202482037883
[ 26 136] 83.9506172839506
[3310] 100.0
average test accuracy: 97.23906717821176
end


In [41]:
#Fold-3 cross validation

X_test=Train_part3
X_train0=np.concatenate((Train_part1[0],Train_part2[0],Train_part4[0],Train_part5[0]),axis=0)
X_train1=np.concatenate((Train_part1[1],Train_part2[1],Train_part4[1],Train_part5[1]),axis=0)
X_train2=np.concatenate((Train_part1[2],Train_part2[2],Train_part4[2],Train_part5[2]),axis=0)
X_train3=np.concatenate((Train_part1[3],Train_part2[3],Train_part4[3],Train_part5[3]),axis=0)
X_train4=np.concatenate((Train_part1[4],Train_part2[4],Train_part4[4],Train_part5[4]),axis=0)
X_train5=np.concatenate((Train_part1[5],Train_part2[5],Train_part4[5],Train_part5[5]),axis=0)
X_train=np.concatenate((X_train0,X_train1,X_train2,X_train3,X_train4,X_train5),axis=0)
gmm = GaussianMixture(n_components=2)
gmm.fit(X_train)

print("Training Accuracies")
labels_rajesh = gmm.predict(X_train0)
a=np.bincount(labels_rajesh.astype(int))

labels_manoj = gmm.predict(X_train1)
b=np.bincount(labels_manoj.astype(int))

labels_vamsi = gmm.predict(X_train2)
c=np.bincount(labels_vamsi.astype(int))

labels_sudheer = gmm.predict(X_train3)
d=np.bincount(labels_sudheer.astype(int))

labels_daiveek=gmm.predict(X_train4)
e=np.bincount(labels_daiveek.astype(int))

labels_imposter= gmm.predict(X_train5)
f=np.bincount(labels_imposter.astype(int))

x=[]
x.append((np.max(a)/np.sum(a))*100)
x.append((np.max(b)/np.sum(b))*100)
x.append((np.max(c)/np.sum(c))*100)
x.append((np.max(d)/np.sum(d))*100)
x.append((np.max(e)/np.sum(e))*100)
x.append((np.max(f)/np.sum(f))*100)

print(a,x[0])
print(b,x[1])
print(c,x[2])
print(d,x[3])
print(e,x[4])
print(f,x[5])
print("average training accuracy:",mean(x))
print("end")
print()

print("Test Accuracies")
labels_rajesh = gmm.predict(X_test[0])
a=np.bincount(labels_rajesh.astype(int))

labels_manoj = gmm.predict(X_test[1])
b=np.bincount(labels_manoj.astype(int))

labels_vamsi = gmm.predict(X_test[2])
c=np.bincount(labels_vamsi.astype(int))

labels_sudheer = gmm.predict(X_test[3])
d=np.bincount(labels_sudheer.astype(int))

labels_daiveek=gmm.predict(X_test[4])
e=np.bincount(labels_daiveek.astype(int))

labels_imposter= gmm.predict(X_test[5])
f=np.bincount(labels_imposter.astype(int))

x=[]
x.append((np.max(a)/np.sum(a))*100)
x.append((np.max(b)/np.sum(b))*100)
x.append((np.max(c)/np.sum(c))*100)
x.append((np.max(d)/np.sum(d))*100)
x.append((np.max(e)/np.sum(e))*100)
x.append((np.max(f)/np.sum(f))*100)

print(a,x[0])
print(b,x[1])
print(c,x[2])
print(d,x[3])
print(e,x[4])
print(f,x[5])
print("average test accuracy:",mean(x))
print("end")
    

Training Accuracies
[7236   38] 99.47759142150123
[15987] 100.0
[12044     8] 99.93362097577165
[12236    15] 99.8775610154273
[490 161] 75.26881720430107
[  400 12844] 96.97976442162489
average training accuracy: 95.25622583977102
end

Test Accuracies
[1811    7] 99.61496149614962
[3996] 100.0
[3009    3] 99.9003984063745
[3060    2] 99.9346832135859
[126  36] 77.77777777777779
[   0 3310] 100.0
average test accuracy: 96.20463681564797
end


In [42]:
#Fold-4 cross validation

X_test=Train_part4
X_train0=np.concatenate((Train_part1[0],Train_part2[0],Train_part3[0],Train_part5[0]),axis=0)
X_train1=np.concatenate((Train_part1[1],Train_part2[1],Train_part3[1],Train_part5[1]),axis=0)
X_train2=np.concatenate((Train_part1[2],Train_part2[2],Train_part3[2],Train_part5[2]),axis=0)
X_train3=np.concatenate((Train_part1[3],Train_part2[3],Train_part3[3],Train_part5[3]),axis=0)
X_train4=np.concatenate((Train_part1[4],Train_part2[4],Train_part3[4],Train_part5[4]),axis=0)
X_train5=np.concatenate((Train_part1[5],Train_part2[5],Train_part3[5],Train_part5[5]),axis=0)
X_train=np.concatenate((X_train0,X_train1,X_train2,X_train3,X_train4,X_train5),axis=0)
gmm = GaussianMixture(n_components=2)
gmm.fit(X_train)

print("Training Accuracies")
labels_rajesh = gmm.predict(X_train0)
a=np.bincount(labels_rajesh.astype(int))

labels_manoj = gmm.predict(X_train1)
b=np.bincount(labels_manoj.astype(int))

labels_vamsi = gmm.predict(X_train2)
c=np.bincount(labels_vamsi.astype(int))

labels_sudheer = gmm.predict(X_train3)
d=np.bincount(labels_sudheer.astype(int))

labels_daiveek=gmm.predict(X_train4)
e=np.bincount(labels_daiveek.astype(int))

labels_imposter= gmm.predict(X_train5)
f=np.bincount(labels_imposter.astype(int))

x=[]
x.append((np.max(a)/np.sum(a))*100)
x.append((np.max(b)/np.sum(b))*100)
x.append((np.max(c)/np.sum(c))*100)
x.append((np.max(d)/np.sum(d))*100)
x.append((np.max(e)/np.sum(e))*100)
x.append((np.max(f)/np.sum(f))*100)

print(a,x[0])
print(b,x[1])
print(c,x[2])
print(d,x[3])
print(e,x[4])
print(f,x[5])
print("average training accuracy:",mean(x))
print("end")
print()

print("Test Accuracies")
labels_rajesh = gmm.predict(X_test[0])
a=np.bincount(labels_rajesh.astype(int))

labels_manoj = gmm.predict(X_test[1])
b=np.bincount(labels_manoj.astype(int))

labels_vamsi = gmm.predict(X_test[2])
c=np.bincount(labels_vamsi.astype(int))

labels_sudheer = gmm.predict(X_test[3])
d=np.bincount(labels_sudheer.astype(int))

labels_daiveek=gmm.predict(X_test[4])
e=np.bincount(labels_daiveek.astype(int))

labels_imposter= gmm.predict(X_test[5])
f=np.bincount(labels_imposter.astype(int))

x=[]
x.append((np.max(a)/np.sum(a))*100)
x.append((np.max(b)/np.sum(b))*100)
x.append((np.max(c)/np.sum(c))*100)
x.append((np.max(d)/np.sum(d))*100)
x.append((np.max(e)/np.sum(e))*100)
x.append((np.max(f)/np.sum(f))*100)

print(a,x[0])
print(b,x[1])
print(c,x[2])
print(d,x[3])
print(e,x[4])
print(f,x[5])
print("average test accuracy:",mean(x))
print("end")
    

Training Accuracies
[7250   24] 99.67005773989553
[15987] 100.0
[12044     8] 99.93362097577165
[12237    14] 99.88572361439883
[506 145] 77.72657450076805
[  400 12844] 96.97976442162489
average training accuracy: 95.69929020874315
end

Test Accuracies
[1813    5] 99.72497249724972
[3996] 100.0
[3010    2] 99.933598937583
[3058    4] 99.86936642717178
[115  47] 70.98765432098766
[   0 3310] 100.0
average test accuracy: 95.0859320304987
end


In [43]:
#Fold-5 cross validation

X_test=Train_part5
X_train0=np.concatenate((Train_part1[0],Train_part2[0],Train_part3[0],Train_part4[0]),axis=0)
X_train1=np.concatenate((Train_part1[1],Train_part2[1],Train_part3[1],Train_part4[1]),axis=0)
X_train2=np.concatenate((Train_part1[2],Train_part2[2],Train_part3[2],Train_part4[2]),axis=0)
X_train3=np.concatenate((Train_part1[3],Train_part2[3],Train_part3[3],Train_part4[3]),axis=0)
X_train4=np.concatenate((Train_part1[4],Train_part2[4],Train_part3[4],Train_part4[4]),axis=0)
X_train5=np.concatenate((Train_part1[5],Train_part2[5],Train_part3[5],Train_part4[5]),axis=0)
X_train=np.concatenate((X_train0,X_train1,X_train2,X_train3,X_train4,X_train5),axis=0)
gmm = GaussianMixture(n_components=2)
gmm.fit(X_train)

print("Training Accuracies")
labels_rajesh = gmm.predict(X_train0)
a=np.bincount(labels_rajesh.astype(int))

labels_manoj = gmm.predict(X_train1)
b=np.bincount(labels_manoj.astype(int))

labels_vamsi = gmm.predict(X_train2)
c=np.bincount(labels_vamsi.astype(int))

labels_sudheer = gmm.predict(X_train3)
d=np.bincount(labels_sudheer.astype(int))

labels_daiveek=gmm.predict(X_train4)
e=np.bincount(labels_daiveek.astype(int))

labels_imposter= gmm.predict(X_train5)
f=np.bincount(labels_imposter.astype(int))

x=[]
x.append((np.max(a)/np.sum(a))*100)
x.append((np.max(b)/np.sum(b))*100)
x.append((np.max(c)/np.sum(c))*100)
x.append((np.max(d)/np.sum(d))*100)
x.append((np.max(e)/np.sum(e))*100)
x.append((np.max(f)/np.sum(f))*100)

print(a,x[0])
print(b,x[1])
print(c,x[2])
print(d,x[3])
print(e,x[4])
print(f,x[5])
print("average training accuracy:",mean(x))
print("end")
print()

print("Test Accuracies")
labels_rajesh = gmm.predict(X_test[0])
a=np.bincount(labels_rajesh.astype(int))

labels_manoj = gmm.predict(X_test[1])
b=np.bincount(labels_manoj.astype(int))

labels_vamsi = gmm.predict(X_test[2])
c=np.bincount(labels_vamsi.astype(int))

labels_sudheer = gmm.predict(X_test[3])
d=np.bincount(labels_sudheer.astype(int))

labels_daiveek=gmm.predict(X_test[4])
e=np.bincount(labels_daiveek.astype(int))

labels_imposter= gmm.predict(X_test[5])
f=np.bincount(labels_imposter.astype(int))

x=[]
x.append((np.max(a)/np.sum(a))*100)
x.append((np.max(b)/np.sum(b))*100)
x.append((np.max(c)/np.sum(c))*100)
x.append((np.max(d)/np.sum(d))*100)
x.append((np.max(e)/np.sum(e))*100)
x.append((np.max(f)/np.sum(f))*100)

print(a,x[0])
print(b,x[1])
print(c,x[2])
print(d,x[3])
print(e,x[4])
print(f,x[5])
print("average test accuracy:",mean(x))
print("end")
    

Training Accuracies
[   5 7267] 99.93124312431243
[    0 15984] 100.0
[    6 12042] 99.95019920318725
[    9 12239] 99.92651861528414
[ 78 570] 87.96296296296296
[12839   401] 96.97129909365559
average training accuracy: 97.45703716656706
end

Test Accuracies
[   3 1817] 99.83516483516483
[   0 3999] 100.0
[   1 3015] 99.96684350132627
[ 527 2538] 82.80587275693311
[ 30 135] 81.81818181818183
[3314] 100.0
average test accuracy: 94.07101048526768
end
